In [3]:
import pandas as pd
from numpy import array, median
import numpy as np
from hw import Jamshidian
from hw import Henrard 
from hw import hw_helper
from hw import calibration as hw_calib
from hw.const import *
from fox_toolbox.utils import xml_parser, rates
from random import choice
import matplotlib.pyplot as plt

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 5.)})
sns.set_style("whitegrid")

%load_ext autoreload
%autoreload 2

### Read IRSM FORM

In [4]:
main_curve, sprds = xml_parser.get_rate_curves(INPUT_5SWO)
dsc_curve = main_curve

try:
    estim_curve = sprds[0]
except TypeError:
    estim_curve = main_curve

cal_basket = list(xml_parser.get_calib_basket(INPUT_5SWO))

### READ IRSM OUT

In [5]:
_, irsmout = xml_parser.get_xml(OUTPUT_5SWO)
ref_swos = list(xml_parser.get_calib_basket(irsmout))

ref_mr, (hw_buckets, hw_sigma) = xml_parser.get_hw_params(irsmout)

ref_sigmas = rates.Curve(hw_buckets, hw_sigma, 'PieceWise')

debug_df = pd.DataFrame()

## Hernard pricer with its ref sigma

In [7]:
calib_premiumsH = []

swo = cal_basket[2]
      
henr_price, debug = Henrard.hw_swo(swo, ref_mr, ref_sigmas, dsc_curve, estim_curve)
debug_df = pd.concat([debug_df, pd.DataFrame(data=debug)], sort=False)
calib_premiumsH.append(henr_price)

a = ref_mr
sigma = ref_sigmas
IsCall = False if swo.pay_rec == 'Receiver' else True
flt_adjs = swo.get_flt_adjustments(dsc_curve, estim_curve)

hw_dates = np.insert(swo.payment_dates, 0, swo.start_date)

coef = Henrard.get_coef(swo.day_count_fractions, swo.strike, flt_adjs)
P_i = Henrard.get_P_i(dsc_curve, hw_dates)
alpha_i, total_var = Henrard.get_alpha_i(a, swo.expiry, hw_dates, sigma)
old_alpha, old_debug = Henrard.old_get_alpha_i(a, swo.expiry, hw_dates, sigma)

calib_premiumsH

[0.015144105523752116]